# Differences in MT and SP in Parasha #3: Lech Lecha (Genesis 12:1-17:27)

## Table of Content <a class="anchor" id="TOC"></a> (ToC)

* <a href="#bullet1">1 - Introduction</a>
* <a href="#bullet2">2 - Load Text-Fabric app and data</a>
* <a href="#bullet3">3 - Compare surface texts of SP and MT</a>
* <a href="#bullet4">4 - Compare texts using minimum Levenshtein distance</a>
* <a href="#bullet5">5 - Comparison of spelling of proper nouns between SP and MT</a>
* <a href="#bullet6">6 - References and acknowledgement</a>
* <a href="#bullet7">7 - Required libraries</a>
* <a href="#bullet8">8 - Notebook version details</a>

# 1 - Introduction <a class="anchor" id="bullet1"></a>
##### [Back to ToC](#TOC)

The Samaritan Pentateuch (SP) is a version of the Torah preserved by the Samaritan community, differing from the Masoretic Text (MT) in several aspects, including language, orthography, and occasionally theological emphasis. This notebook compares the text of the Masoretic Text, based on the BHSA dataset in Text-Fabric, with the Samaritan Pentateuch, also available as a Text-Fabric dataset.<a href="#ref1"><sup>1</sup></a>

In this analysis, we focus on comparing the text of the verses in a specific parasha, highlighting differences in wording and orthography. Additionally, special attention is given to spelling variations of proper nouns between the two traditions. This notebook draws inspiration from the notebook provided by Martijn Naaijer<a href="#ref2"><sup>2</sup></a> and aims to explore the textual nuances between these two important versions of the Torah.

# 2 - Load Text-Fabric app and data <a class="anchor" id="bullet2"></a>
##### [Back to ToC](#TOC)

The following code will load the Text-Fabric version of the [Samaritan Pentatuch](https://github.com/DT-UCPH/sp), the [Biblia Hebraica Stuttgartensia (Amstelodamensis)](https://etcbc.github.io/bhsa/) together with the additonal parasha related features from [tonyjurg/BHSaddons](https://github.com/tonyjurg/BHSaddons).

In [1]:
from tf.app import use

# Load the SP data, and rename the node features class F,
# the locality class L and the text class T, 
# then they cannot be overwritten while loading the MT.
SP = use('DT-UCPH/sp', version='3.4')
Fsp, Lsp, Tsp = SP.api.F, SP.api.L, SP.api.T

# Do the same for the MT dataset (BHSA) together with BHSaddons 
MT = use('etcbc/bhsa', version='2021',mod="tonyjurg/BHSaddons/tf/")
Fmt, Lmt, Tmt = MT.api.F, MT.api.L, MT.api.T

**Locating corpus resources ...**

Name,# of nodes,# slots / node,% coverage
book,5,79878.40,100
chapter,187,2135.79,100
verse,5841,68.38,100
word,114890,3.48,100
sign,399392,1.00,100


**Locating corpus resources ...**

Name,# of nodes,# slots / node,% coverage
book,39,10938.21,100
chapter,929,459.19,100
lex,9230,46.22,100
verse,23213,18.38,100
half_verse,45179,9.44,100
sentence,63717,6.70,100
sentence_atom,64514,6.61,100
clause,88131,4.84,100
clause_atom,90704,4.70,100
phrase,253203,1.68,100


# 3 - Compare surface texts of SP and MT <a class="anchor" id="bullet3"></a>
##### [Back to ToC](#TOC)

In this section, we compare the surface texts of the Samaritan Pentateuch (SP) and the Masoretic Text (MT) at the verse level. By analyzing the wording and structure of these texts, we aim to identify variations.

In [2]:
# find all word nodes for this parasha (we can either use the transliterated name or the sequence number)
parashaQuery = '''
verse parashanum=3
'''
parashaResults = MT.search(parashaQuery)

  0.01s 126 results


In [3]:
# Extract book, chapter, and verse information
bookChapterVerseList = [
    Tmt.sectionFromNode(verse[0]) for verse in parashaResults
]

# Store parashname, start and end verse for future use
startNode=parashaResults[0][0]
endNode=parashaResults[-1][0]
parashaNameHebrew=Fmt.parashahebr.v(startNode)
parashaNameEnglish=Fmt.parashatrans.v(startNode)
bookStart,chapterStart,startVerse=Tmt.sectionFromNode(startNode)
parashaStart=f'{bookStart} {chapterStart}:{startVerse}'
bookEnd,chapterEnd,startEnd=Tmt.sectionFromNode(endNode)
parashaEnd=f'{chapterEnd}:{startEnd}'

In [4]:
# Function to reconstruct verses
def reconstructVerses(F, L, T, textFeature, inputList):
    """Reconstruct text for each verse."""
    verseTexts = {}
    for verseName in inputList:
        verseText = ''
        verseNode = T.nodeFromSection(verseName)
        wordNodes = L.d(verseNode, 'word')
        for wordNode in wordNodes:
            wordText = eval(f'F.{textFeature}.v(wordNode)')
            trailer = F.trailer.v(wordNode)
            if wordText:
                verseText += wordText + (trailer if trailer else ' ')
        verseTexts[verseName] = verseText.strip()
    return verseTexts
    
SPverses = reconstructVerses(Fsp, Lsp, Tsp, 'g_cons', bookChapterVerseList)
MTverses = reconstructVerses(Fmt, Lmt, Tmt, 'g_cons', bookChapterVerseList)

In [5]:
from difflib import SequenceMatcher

def highlightMatches(baseText, comparisonText):
    matcher = SequenceMatcher(None, baseText, comparisonText)
    highlightedComparisonText = "" 
    for tag, i1, i2, j1, j2 in matcher.get_opcodes():
        if tag == "equal":  # Identical parts
            highlightedComparisonText += comparisonText[j1:j2]
        else:  # Non-matching parts
            highlightedComparisonText += f'<mark>{comparisonText[j1:j2]}</mark>'  
    return highlightedComparisonText

def cleanText(text):
    replacements = [
         # for the transcoded strings
         ('00_P', ''),  # Remove '00_P'
         ('00_S', ''),  # Remove '00_S'
         ('00', ''),    # Remove '00'
         ('&', ' '),    # Replace '&' with a space
         # for the Hebrew strings
         ('ס ', ''),    # Final Samekh
         ('פ ', ''),    # Final Pe
         ('׃', ''),     # End of verse
         ('־',' ')      # maqaf
    ]
    # Apply each replacement
    for old, new in replacements:
        text = text.replace(old, new)
    return text

# Function to format and highlight verse differences between MT and SP
def formatAndHighlight(label, MTverseText, SPverseText):
    book, chapter, verse = label
    MTverseNode = Tmt.nodeFromSection(label)
    MTtext = cleanText(Tmt.text(MTverseNode, "text-orig-plain"))
    SPverseNode = Tsp.nodeFromSection(label)
    SPtext = Tsp.text(SPverseNode)
    SPmarkedText = highlightMatches(MTtext, SPtext)
    MTmarkedText = highlightMatches(SPtext, MTtext)
    formattedDiff = (
        f'<h4>{book} {chapter}:{verse}</h4>'
        f'<p><b>SP:</b> {SPmarkedText}<br><b>MT:</b> {MTmarkedText}</p>'
    )
    return formattedDiff

# Create header
MT.dm(f'### Differences between MT and SP for parasha {parashaNameEnglish} ({parashaStart}-{parashaEnd})')

# Highlight and display differences for SP and MT verses
for label, MTverseText in MTverses.items():
    SPverseText = SPverses.get(label, '')
    MTverseText=cleanText(MTverseText)
    if MTverseText != SPverseText:  # Check for differences
        difference = formatAndHighlight(label, MTverseText, SPverseText)
        MT.dm(difference)

### Differences between MT and SP for parasha Lech Lecha (Genesis 12:1-17:27)

<h4>Genesis 12:2</h4><p><b>SP:</b> ואעש<mark></mark>ך לגוי גדול ואברכך ואגדלה ש<mark></mark>מך וה<mark>ו</mark>י<mark></mark> ברכה <br><b>MT:</b> ואעש<mark>ׂ</mark>ך לגוי גדול ואברכך ואגדלה ש<mark>ׁ</mark>מך וה<mark></mark>י<mark>ה</mark> ברכה </p>

<h4>Genesis 12:3</h4><p><b>SP:</b> ואבר<mark>ך</mark> מברכיך ומקלל<mark>י</mark>ך אאר ונברכו בך כל מש<mark></mark>פח<mark>ו</mark>ת האדמה <br><b>MT:</b> ואבר<mark>כה</mark> מברכיך ומקלל<mark></mark>ך אאר ונברכו בך כל מש<mark>ׁ</mark>פח<mark></mark>ת האדמה </p>

<h4>Genesis 12:6</h4><p><b>SP:</b> ויעבר אברם בארץ עד מקום ש<mark></mark>כם עד אלון מור<mark>א</mark> והכנעני אז בארץ <br><b>MT:</b> ויעבר אברם בארץ עד מקום ש<mark>ׁ</mark>כם עד אלון מור<mark>ה</mark> והכנעני אז בארץ </p>

<h4>Genesis 12:7</h4><p><b>SP:</b> וירא יהוה אל אברם ויאמר<mark> לו</mark> לזרעך אתן את הארץ הזאת ויבן ש<mark></mark>ם מזבח ליהוה הנראה אליו <br><b>MT:</b> וירא יהוה אל אברם ויאמר<mark></mark> לזרעך אתן את הארץ הזאת ויבן ש<mark>ׁ</mark>ם מזבח ליהוה הנראה אליו </p>

...

<h4>Genesis 17:19</h4><p><b>SP:</b> ויאמר אלהים אבל <mark>הנה </mark>ש<mark></mark>רה אש<mark></mark>תך ילדת לך בן וקראת את ש<mark></mark>מו יצחק והקמתי את בריתי אתו לברית עולם <mark>ו</mark>לזרעו אחריו <br><b>MT:</b> ויאמר אלהים אבל <mark></mark>ש<mark>ׂ</mark>רה אש<mark>ׁ</mark>תך ילדת לך בן וקראת את ש<mark>ׁ</mark>מו יצחק והקמתי את בריתי אתו לברית עולם <mark></mark>לזרעו אחריו </p>

<h4>Genesis 17:20</h4><p><b>SP:</b> וליש<mark></mark>מעאל ש<mark></mark>מעתיך הנה<mark></mark> ברכתי אתו והפריתי אתו והרביתי אתו במאד מאד ש<mark></mark>נים עש<mark></mark>ר נש<mark></mark>יא<mark>י</mark>ם יוליד ונתתיו לגוי גדול <br><b>MT:</b> וליש<mark>ׁ</mark>מעאל ש<mark>ׁ</mark>מעתיך הנה<mark>׀</mark> ברכתי אתו והפריתי אתו והרביתי אתו במאד מאד ש<mark>ׁ</mark>נים עש<mark>ׂ</mark>ר נש<mark>ׂ</mark>יא<mark></mark>ם יוליד ונתתיו לגוי גדול </p>

<h4>Genesis 17:24</h4><p><b>SP:</b> ואברהם בן תש<mark></mark>עים ותש<mark></mark>ע ש<mark></mark>נ<mark>ים</mark> בהמלו <mark>את </mark>בש<mark></mark>ר ערלתו <br><b>MT:</b> ואברהם בן תש<mark>ׁ</mark>עים ותש<mark>ׁ</mark>ע ש<mark>ׁ</mark>נ<mark>ה</mark> בהמלו <mark></mark>בש<mark>ׂ</mark>ר ערלתו </p>

<h4>Genesis 17:26</h4><p><b>SP:</b> בעצם היום הזה נמ<mark></mark>ל אברהם ויש<mark></mark>מעאל בנו <br><b>MT:</b> בעצם היום הזה נמ<mark>ו</mark>ל אברהם ויש<mark>ׁ</mark>מעאל בנו </p>

<h4>Genesis 17:27</h4><p><b>SP:</b> וכל אנש<mark></mark>י ביתו יליד בית ומקנת כסף מאת בן נכר נמלו אתו <br><b>MT:</b> וכל אנש<mark>ׁ</mark>י ביתו יליד בית ומקנת כסף מאת בן נכר נמלו אתו </p>

# 4 - Compare texts using minimum Levenshtein distance<a class="anchor" id="bullet4"></a>
##### [Back to ToC](#TOC)

The Levenshtein distance measures the minimum number of single-character edits (insertions, deletions, or substitutions) needed to transform one text into another, providing a quantitative way to compare textual differences. For comparing the Masoretic Text and Samaritan Pentateuch, it highlights variations in spelling, word order, or minor textual changes. 
In the context of the Levenshtein distance (in the script below `threshold`), a higher number indicates greater dissimilarity between two texts, meaning more edits (insertions, deletions, or substitutions) are needed to transform one text into the other.

In [6]:
# Compare verses based on Levenshtein distance
from Levenshtein import distance
threshold = 20

# Create header
MT.dm(f'### Levenshtein distance {threshold} between MT and SP for parasha {parashaNameEnglish} ({parashaStart}-{parashaEnd})')

for label, MTverseText in MTverses.items():
    SPverseText = SPverses.get(label, '')
    if distance(MTverseText, SPverseText) > threshold:
        formattedDiff = formatAndHighlight(label, MTverseText, SPverseText)
        MT.dm(formattedDiff)

### Levenshtein distance 20 between MT and SP for parasha Lech Lecha (Genesis 12:1-17:27)

<h4>Genesis 12:16</h4><p><b>SP:</b> ולאברם הי<mark>י</mark>ט<mark></mark>ב בעבורה ויהי לו צאן ובקר <mark>מקנה כבד מאד עבדים ושפחות </mark>וחמרים ו<mark></mark>אתנ<mark>ו</mark>ת וגמלים <br><b>MT:</b> ולאברם הי<mark>ט</mark>י<mark></mark>ב בעבורה ויהי לו צאן ובקר <mark>וח</mark>מ<mark>רים</mark> <mark>ו</mark>עבדים וש<mark>ׁ</mark>פח<mark></mark>ת<mark></mark> ואתנ<mark></mark>ת וגמלים </p>

<h4>Genesis 17:14</h4><p><b>SP:</b> וערל<mark></mark> זכר אש<mark></mark>ר לא ימול את בש<mark></mark>ר ערלתו<mark> ביום השמיני</mark> ונכרתה הנפש<mark></mark> הה<mark>י</mark>א מעמיה את בריתי הפר <br><b>MT:</b> וערל<mark>׀</mark> זכר אש<mark>ׁ</mark>ר לא ימול את בש<mark>ׂ</mark>ר ערלתו<mark></mark> ונכרתה הנפש<mark>ׁ</mark> הה<mark>ו</mark>א מעמיה את בריתי הפר </p>

# 5 - Comparison of spelling of proper nouns between SP and MT<a class="anchor" id="bullet5"></a>
##### [Back to ToC](#TOC)

This section focuses on comparing the spelling of proper nouns between the Samaritan Pentateuch (SP) and the Masoretic Text (MT). Proper nouns, including names of people, places, and unique terms, often exhibit variations in spelling

In [7]:
import collections

def collectProperNounSpellings(F, L, T, inputList):
    """
    Collect proper noun spellings and their associated word node numbers.
    Ensures only one tuple is stored for each lexeme-to-spelling mapping.
    """
    properNounsSpellings = {}
    for bookChapterVerse in inputList:
        verseNode = T.nodeFromSection(bookChapterVerse)
        wordNodes = L.d(verseNode, 'word')
        for wordNode in wordNodes:
            if F.sp.v(wordNode) == 'nmpr':  # Check if the word is a proper noun
                lex = F.lex.v(wordNode)    # Lexical form
                spelling = F.g_cons.v(wordNode)  # Spelling
                # Store only the first occurrence for each lex-to-cons mapping
                if lex not in properNounsSpellings or spelling not in {item[0] for item in properNounsSpellings[lex]}:
                    properNounsSpellings.setdefault(lex, []).append((spelling, wordNode))
    return properNounsSpellings
        
SPspellingDict = collectProperNounSpellings(Fsp, Lsp, Tsp, bookChapterVerseList) 
MTspellingDict = collectProperNounSpellings(Fmt, Lmt, Tmt, bookChapterVerseList)

In [8]:
# Create header
MT.dm(f'### Spelling differences proper nouns in SP and MT for parasha {parashaNameEnglish} ({parashaStart}-{parashaEnd})')
    
for lex, MTspellings in MTspellingDict.items():
    # Retrieve SP spellings, defaulting to an empty set if lex is not found
    SPspellings = SPspellingDict.get(lex, set())

    # Extract only the spellings (ignoring node numbers) for comparison
    MTspellingSet = {spelling for spelling, _ in MTspellings}
    SPspellingSet = {spelling for spelling, _ in SPspellings}

    # Compare the sets of spellings
    if MTspellingSet != SPspellingSet:
        # Print MT spelling with reference
        MTnode = list(MTspellings)[0][1]  # Get first tuple's node number
        book, chapter, verse = Tmt.sectionFromNode(MTnode)
        MTgloss = Fmt.gloss.v(MTnode)
        MTspelling = Fmt.g_cons_utf8.v(MTnode)

        # Build Markdown output
        output = f"#### Word: **{MTgloss}** ({book} {chapter}:{verse})\n"
        output += f"- **MT Spelling:** `{MTspelling}`\n"

        # Print SP spellings with reference
        if SPspellings:
            SPnode = list(SPspellings)[0][1]  # Get first tuple's node number
            SPspelling = Fsp.g_cons_utf8.v(SPnode)
            output += f"- **SP Spelling:** `{SPspelling}`\n"
        else:
            output += "- **SP Spelling:** None\n"

        # Print the formatted output
        MT.dm(output)

### Spelling differences proper nouns in SP and MT for parasha Lech Lecha (Genesis 12:1-17:27)

#### Word: **Moreh** (Genesis 12:6)
- **MT Spelling:** `מורה`
- **SP Spelling:** `מורא`


#### Word: **Zoar** (Genesis 13:10)
- **MT Spelling:** `צער`
- **SP Spelling:** `צערה`


#### Word: **Kedorlaomer** (Genesis 14:1)
- **MT Spelling:** `כדרלעמר`
- **SP Spelling:** `כדר לעמר`


#### Word: **Shemeber** (Genesis 14:2)
- **MT Spelling:** `שׁמאבר`
- **SP Spelling:** `שׁמ אבד`


...

#### Word: **Hazazon Tamar** (Genesis 14:7)
- **MT Spelling:** `חצצן תמר`
- **SP Spelling:** `חצצונ תמר`


#### Word: **Eshcol** (Genesis 14:13)
- **MT Spelling:** `אשׁכל`
- **SP Spelling:** `אשׁכול`


#### Word: **Aner** (Genesis 14:13)
- **MT Spelling:** `ענר`
- **SP Spelling:** `ענרם`


#### Word: **Shaveh** (Genesis 14:17)
- **MT Spelling:** `שׁוה`
- **SP Spelling:** `שׁוי`


#### Word: **Beer Lahai Roi** (Genesis 16:14)
- **MT Spelling:** `לחי ראי`
- **SP Spelling:** `לחי ראה`


# 6 - References and acknowledgement <a class="anchor" id="bullet6"></a>
##### [Back to ToC](#TOC)

<a class="anchor" id="bullet1"><sup>1</sup></a> Christian Canu Højgaard, Martijn Naaijer, & Stefan Schorch. (2023). Text-Fabric Dataset of the Samaritan Pentateuch. Zenodo. https://doi.org/10.5281/zenodo.7734632

<a class="anchor" id="bullet2"><sup>2</sup> [Notebook created by Martijn Naaijer](https://github.com/DT-UCPH/sp/blob/main/notebooks/combine_sp_with_mt_data.ipynb)

# 7 - Required libraries <a class="anchor" id="bullet7"></a>
##### [Back to ToC](#TOC)

The scripts in this notebook require (beside `text-fabric`) the following Python libraries to be installed in the environment:

    collections
    difflib
    Levenshtein

You can install any missing library from within Jupyter Notebook using either`pip` or `pip3`.

# 8 - Notebook version details<a class="anchor" id="bullet8"></a>
##### [Back to ToC](#TOC)

<div style="float: left;">
  <table>
    <tr>
      <td><strong>Author</strong></td>
      <td>Tony Jurg</td>
    </tr>
    <tr>
      <td><strong>Version</strong></td>
      <td>1.0</td>
    </tr>
    <tr>
      <td><strong>Date</strong></td>
      <td>17 November 2024</td>
    </tr>
  </table>
</div>